In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
from postprocessor.core.processes.fft import fft
from postprocessor.core.processes.findpeaks import findpeaks
from postprocessor.routines.mean_plot import mean_plot

from postprocessor.core.multisignal.crosscorr import crosscorr
# from src.crosscorr import crosscorr
from src.synthetic import (
    fitzhugh_nagumo,
    fitzhugh_nagumo_stochastic,
    gillespie_noise,
    gillespie_noise_raw,
    harmonic,
    harmonic_stochastic,
    sinusoid,
)
from src.utils import multiarray_random_shift, simple_median_plot, tile_signals

GENERATE SIGNALS

Step 1: Define parameters

Step 2: Generate arrays of signals

In [ ]:
num_timeseries = 100
timeaxis = np.linspace(0, 500, 500)

Choice group A: fill with same signal
(thus using numpy.tile instead of for loop to make it fast)

Choice 1: Array of FHNs

In [ ]:
fitzhugh_nagumo_single, _ = fitzhugh_nagumo(
    timeaxis=timeaxis, ext_stimulus=0.4, tau=12.5, a=0.7, b=0.82
)
fitzhugh_nagumo_single -= np.mean(fitzhugh_nagumo_single)
fitzhugh_nagumo_array = tile_signals([fitzhugh_nagumo_single], [num_timeseries])

Choice 2: Array of sinusoids

In [ ]:
sinusoid_single = sinusoid(timeaxis=timeaxis, amp=1, freq=0.0235, phase=0)
sinusoid_array = tile_signals([sinusoid_single], [num_timeseries])

Choice 3: Mixed array of sinusoids

In [ ]:
sinusoid_long = sinusoid(timeaxis=timeaxis, amp=1, freq=0.03, phase=0)
sinusoid_short = sinusoid(timeaxis=timeaxis, amp=1, freq=0.04, phase=0)
sinusoid_mixed_array = tile_signals([sinusoid_short, sinusoid_long], [20, 20])

Shift phases -- grouping pairs/triplets/tuples of signals that come from
the same sources

In [ ]:
fitzhugh_nagumo_array, sinusoid_array = multiarray_random_shift(
    [fitzhugh_nagumo_array, sinusoid_array]
)

In [ ]:
sinusoid_array = multiarray_random_shift([sinusoid_array])[0]

In [ ]:
sinusoid_mixed_array = multiarray_random_shift([sinusoid_mixed_array])[0]

Choice group B: each row is different

Choice 4: Array of sinusoids, random phases

In [ ]:
sinusoid_outofphase_array = np.empty((num_timeseries, len(timeaxis)), dtype=float)
for row_index in range(num_timeseries):
    phase = np.random.random() * 2 * np.pi
    sinusoid_outofphase_array[row_index] = sinusoid(
        timeaxis=timeaxis, amp=1, freq=0.03, phase=phase
    )

Choice 5: Mixed array of sinusoids, random phases

In [ ]:
def generate_sinusoid_outofphase_array(num_timeseries, timeaxis, freq):
    sinusoid_outofphase_array = np.empty((num_timeseries, len(timeaxis)), dtype=float)
    for row_index in range(num_timeseries):
        phase = np.random.random() * 2 * np.pi
        sinusoid_outofphase_array[row_index] = sinusoid(
            timeaxis=timeaxis, amp=1, freq=freq, phase=phase
        )
    return sinusoid_outofphase_array


sinusoid_outofphase_long = generate_sinusoid_outofphase_array(200, timeaxis, 0.03)
sinusoid_outofphase_short = generate_sinusoid_outofphase_array(200, timeaxis, 0.04)
sinusoid_mixed_array = np.concatenate(
    (sinusoid_outofphase_short, sinusoid_outofphase_long)
)

Step 3: Generate arrays of noise

Choice 1: white/Gaussian noise

In [ ]:
white_noise_std = 3
white_noise_array1 = np.random.normal(
    loc=0, scale=white_noise_std, size=(num_timeseries, len(timeaxis))
)
white_noise_array2 = np.random.normal(
    loc=0, scale=white_noise_std, size=(num_timeseries, len(timeaxis))
)

Choice 2: Gillespie noise

In [ ]:
# SET PARAMETERS
noise_timescale = 20
noise_amp = 100
gill_time_final = 7500
gill_num_intervals = 5000

In [ ]:
# GENERATE noise array
gill_noise_array = gillespie_noise(
    num_timeseries=num_timeseries,
    num_timepoints=len(timeaxis),
    noise_timescale=noise_timescale,
    noise_amp=noise_amp,
    time_final=gill_time_final,
    grid_num_intervals=gill_num_intervals,
)

In [ ]:
# Filename generator
deathrate = 1 / noise_timescale
birthrate = noise_amp / noise_timescale
deathrate_str = str(deathrate).replace(".", "p")
birthrate_str = str(birthrate).replace(".", "p")
gill_noise_filename = "gillespienoise_k" + birthrate_str + "_d" + deathrate_str + ".csv"

In [ ]:
# LOAD noise array
gill_noise_array = np.genfromtxt(gill_noise_filename, delimiter=",")

In [ ]:
# SAVE noise array
np.savetxt(gill_noise_filename, gill_noise_array, delimiter=",")

Step 4: Assign signal and noisy arrays, then construct final dataframes

Step 4.1: Assign signal arrays

In [ ]:
signal_array1 = sinusoid_array
signal_array2 = fitzhugh_nagumo_array

Step 4.2: Assign noise arrays

In [ ]:
noise_array1 = gill_noise_array

noise_array2 = gill_noise_array.copy()
rng = np.random.default_rng()
rng.shuffle(noise_array2)

Step 4.3: Add signal and noise

In [ ]:
signal_array1 = signal_array1 #+ noise_array1
signal_array2 = signal_array2 #+ noise_array2

Step 4.4: Construct dataframes for correlation processes

In [ ]:
signal_df1 = pd.DataFrame(signal_array1)
signal_df2 = pd.DataFrame(signal_array2)

Step 5: Autocorrelation & cross-correlation

Autocorrelation

In [ ]:
autocorr_result = crosscorr.as_function(
    signal_df1, stationary=False, normalised=True, only_pos=True
)

Cross-correlation

In [ ]:
crosscorr_result = crosscorr.as_function(signal_df1, signal_df2)

Mean across replicates

In [ ]:
mean_across_replicates = np.nanmean(signal_array1, axis=0).reshape(
    (1, signal_array1.shape[1])
)
mean_across_replicates = mean_across_replicates.T

PLOTTING

input data

In [ ]:
sns.heatmap(signal_df1)

In [ ]:
sns.heatmap(signal_df2)

gillespie noise

In [ ]:
gill_array = signal_array1
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(
    mean_across_replicates,
    linewidth=3,
    label=f"mean across {gill_array.shape[0]} replicates",
)
ax.plot(gill_array[0], linewidth=1, label="replicate 1")
ax.plot(gill_array[1], linewidth=1, label="replicate 2")
ax.set_xlabel("Time point")
ax.set_ylabel("Signal")
fig.legend()

acf of signals with gillespie noise, with options

In [ ]:
fit_exp_decay = True
scale_lag_by_pd = True
freq = 0.03

fig, ax = plt.subplots()

# scale lag axis by sinusoid period
autocorr_result_scaled = autocorr_result.copy()
if scale_lag_by_pd:
    freq = freq
    xlabel = "Lag (periods)"
    plt.vlines(x=[1, 2, 3, 4], ymin=-1, ymax=1, ls="--")
else:
    freq = 1
    xlabel = "Lag (time points)"
autocorr_result_scaled.columns *= freq

# fit exp decay
if fit_exp_decay:
    decayrate = (gill_time_final / (gill_num_intervals - 1)) * (1 / noise_timescale)
    t = autocorr_result.columns.to_numpy()
    decay_function = np.exp(-(decayrate) * t)
    ax.plot(t * freq, decay_function, color="r")

# draw acf
simple_median_plot(
    autocorr_result_scaled,
    xlabel=xlabel,
    ylabel="Autocorrelation function",
    ax=ax,
)
# and axes
plt.axhline(0, color="k")
plt.axvline(0, color="k")

at longer lags, fewer data points are used to compute acf, and thus the std dev across replicate acfs at those points are greater

In [ ]:
lag = np.linspace(0, 499, 500)
num_datapoints = signal_array1.shape[1] * np.linspace(500, 1, 500)
acf_variation = np.std(autocorr_result)

fig, ax = plt.subplots()
ax.plot(num_datapoints, acf_variation)
ax.set_xlabel("Number of data points used at lag value")
ax.set_ylabel("Standard deviation of\nautocorrelation function values at lag value")

variation between acfs, expressed as area the curve of std dev change over lag time

In [ ]:
std_array = autocorr_result.std(axis=0)
plt.plot(std_array)
auc = np.trapz(std_array)
print(auc)

envelope function

In [ ]:
# find peaks & troughs
mean_acf_df = autocorr_result.mean().to_frame().T
peaks_df = findpeaks.as_function(mean_acf_df)
troughs_df = findpeaks.as_function(-mean_acf_df)

# datatype conversions
timeaxis = mean_acf_df.columns.to_numpy()
mean_acf = mean_acf_df.to_numpy()[0]
peaks_mask = peaks_df.to_numpy()[0] != 0
troughs_mask = troughs_df.to_numpy()[0] != 0

In [ ]:
# https://stackoverflow.com/questions/3938042/fitting-exponential-decay-with-no-initial-guessing
def model_func(t, A, K, C):
    return A * np.exp(-K * t) + C


def fit_exp_linear(t, y, C=0):
    y = y - C
    y = np.log(y)
    K, A_log = np.polyfit(t, y, 1)
    A = np.exp(A_log)
    return A, K


def fit_exp_nonlinear(t, y, p0):
    opt_parms, parm_cov = sp.optimize.curve_fit(model_func, t, y, p0, maxfev=1000)
    A, K, C = opt_parms
    return A, K, C


# initial guess is the decay function in acf plot
central_decay_rate = (gill_time_final / (gill_num_intervals - 1)) * (
    1 / noise_timescale
)
initial_A = 1
initial_K = central_decay_rate
initial_C = 0
initial_guess = [initial_A, initial_K, initial_C]

# fit peaks
upper_A, upper_K, upper_C = fit_exp_nonlinear(
    timeaxis[peaks_mask],
    mean_acf[peaks_mask],
    p0=initial_guess,
)
upper_func = model_func(timeaxis, upper_A, upper_K, upper_C)

# fit troughs
lower_A, lower_K, lower_C = fit_exp_nonlinear(
    timeaxis[troughs_mask],
    -mean_acf[troughs_mask],
    p0=initial_guess,
)
lower_func = -model_func(timeaxis, lower_A, lower_K, lower_C)

In [ ]:
plt.plot(timeaxis, mean_acf)
plt.scatter(timeaxis[peaks_mask], mean_acf[peaks_mask])
plt.scatter(timeaxis[troughs_mask], mean_acf[troughs_mask])
plt.plot(timeaxis, upper_func)
plt.plot(timeaxis, lower_func)

print(f"upper envelope: {upper_A:.4f} * exp(- {upper_K:.4f}) + {upper_C:.4f}")
print(f"lower envelope: {lower_A:.4f} * exp(- {lower_K:.4f}) + {lower_C:.4f}")

cross-correlation

In [ ]:
ts1 = signal_df1.iloc[0].to_numpy()
ts2 = signal_df2.iloc[0].to_numpy()

tps = signal_df1.columns.to_numpy()
signal_df1_peaks = findpeaks.as_function(signal_df1)
signal_df2_peaks = findpeaks.as_function(signal_df2)
peaks1 = signal_df1_peaks.iloc[0].to_numpy() == 1
peaks2 = signal_df2_peaks.iloc[0].to_numpy() == 1

fig, ax = plt.subplots(figsize=(12,4))

ax.plot(ts1, label='sinusoid')
ax.plot(ts2, label='FitzHugh-Nagumo')

ax.scatter(tps[peaks1], ts1[peaks1])
ax.scatter(tps[peaks2], ts2[peaks2])

ax.set_xlabel("Time point")
ax.set_ylabel("Signal")

fig.legend()

In [ ]:
tps[peaks1]

In [ ]:
tps[peaks2]

In [ ]:
np.mean(tps[peaks2] - tps[peaks1])

In [ ]:
simple_median_plot(
    crosscorr_result, ylabel="Cross correlation", xlabel="Lag (time points)"
)
plt.axhline(0, color="k")
plt.axvline(0, color="k")

In [ ]:
lags = crosscorr_result.columns.to_numpy()

In [ ]:
xcf_peaks_df = findpeaks.as_function(crosscorr_result.median().to_frame().T)

In [ ]:
xcf_peaks = xcf_peaks_df.to_numpy().ravel() == 1

In [ ]:
lags[xcf_peaks]